In [53]:
import re
import numpy as np
from PIL import Image
import cv2
import random

from keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import InputLayer, Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop

import tensorflow as tf

In [54]:
epochs = 10
batch_size = 16
margin = 1
img_height = 256
img_width = 256

In [55]:
import pathlib
train_dir = pathlib.Path('E:/phd/datasetSiameseShort/train')
test_dir = pathlib.Path('E:/phd/datasetSiameseShort/test')
# train_dir = pathlib.Path('./batch')
# test_dir = pathlib.Path('./batch_test')

In [56]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1970 files belonging to 197 classes.


In [57]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 250 files belonging to 25 classes.


In [58]:
class_names = train_ds.class_names
print(class_names)
output_classes = len(class_names)

['S', 'S()', 'S(10)', 'S(100)', 'S(101)', 'S(102)', 'S(103)', 'S(104)', 'S(105)', 'S(106)', 'S(107)', 'S(108)', 'S(109)', 'S(11)', 'S(110)', 'S(111)', 'S(112)', 'S(113)', 'S(114)', 'S(115)', 'S(116)', 'S(117)', 'S(118)', 'S(119)', 'S(12)', 'S(120)', 'S(121)', 'S(122)', 'S(123)', 'S(124)', 'S(125)', 'S(126)', 'S(127)', 'S(128)', 'S(129)', 'S(13)', 'S(130)', 'S(131)', 'S(132)', 'S(133)', 'S(134)', 'S(135)', 'S(136)', 'S(137)', 'S(138)', 'S(139)', 'S(14)', 'S(140)', 'S(141)', 'S(142)', 'S(143)', 'S(144)', 'S(145)', 'S(146)', 'S(147)', 'S(148)', 'S(149)', 'S(15)', 'S(150)', 'S(151)', 'S(152)', 'S(153)', 'S(154)', 'S(155)', 'S(156)', 'S(157)', 'S(158)', 'S(159)', 'S(16)', 'S(160)', 'S(161)', 'S(162)', 'S(163)', 'S(164)', 'S(165)', 'S(166)', 'S(167)', 'S(168)', 'S(169)', 'S(17)', 'S(170)', 'S(171)', 'S(172)', 'S(173)', 'S(174)', 'S(175)', 'S(176)', 'S(177)', 'S(178)', 'S(179)', 'S(18)', 'S(180)', 'S(181)', 'S(182)', 'S(183)', 'S(184)', 'S(185)', 'S(186)', 'S(187)', 'S(189)', 'S(19)', 'S(190)

In [59]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [60]:
train_wholeimage = []
train_wholelabel = []

for images, labels in train_ds.unbatch().take(-1):
    train_wholeimage.append(images)
    train_wholelabel.append(labels)
train_wholelabel= np.array(train_wholelabel, dtype=np.float32)
train_wholeimage = np.array(train_wholeimage, dtype=np.float32)

In [61]:
val_wholeimage = []
val_wholelabel = []

for images, labels in val_ds.unbatch().take(-1):
    val_wholeimage.append(images)
    val_wholelabel.append(labels)
val_wholelabel= np.array(val_wholelabel, dtype=np.float32)
val_wholeimage = np.array(val_wholeimage, dtype=np.float32)

In [62]:
def make_pairs(x, y):
    """Creates a tuple containing image pairs with corresponding label.

    Arguments:
        x: List containing images, each index in this list corresponds to one image.
        y: List containing labels, each label with datatype of `int`.

    Returns:
        Tuple containing two numpy arrays as (pairs_of_samples, labels),
        where pairs_of_samples' shape is (2len(x), 2,n_features_dims) and
        labels are a binary array of shape (2len(x)).
    """

    num_classes = max(y) + 1
    digit_indices = [np.where(y == i)[0] for i in range(num_classes)]

    pairs = []
    labels = []

    for idx1 in range(len(x)):
        # add a matching example
        x1 = x[idx1]
        label1 = y[idx1]
        idx2 = random.choice(digit_indices[label1])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [1]

        # add a non-matching example
        label2 = random.randint(0, num_classes - 1)
        while label2 == label1:
            label2 = random.randint(0, num_classes - 1)

        idx2 = random.choice(digit_indices[label2])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [0]

    return np.array(pairs), np.array(labels).astype("float32")

In [63]:
train_wholelabel = train_wholelabel.astype("uint8")
val_wholelabel = val_wholelabel.astype("uint8")

In [64]:
# make train pairs
pairs_train, labels_train = make_pairs(train_wholeimage, train_wholelabel)
# make validation pairs
pairs_val, labels_val = make_pairs(val_wholeimage, val_wholelabel)

In [65]:
x_train_1 = pairs_train[:, 0]
x_train_2 = pairs_train[:, 1]

x_val_1 = pairs_val[:, 0]
x_val_2 = pairs_val[:, 1]

IMG_SHAPE = x_train_1.shape[1:]

In [66]:
# input_layer = tf.keras.Input(shape=IMG_SHAPE)
# batchnormalizelayer = tf.keras.layers.BatchNormalization()
# base_model = tf.keras.applications.Xception(include_top=False,
#                                                weights='imagenet')
# base_model.trainable = False
# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# prediction_layer = tf.keras.layers.Dense(40, activation="tanh")

# embedding_network = tf.keras.Sequential([
#   input_layer,
#   batchnormalizelayer,
#   base_model,
#   global_average_layer,
#   prediction_layer
# ])

# embedding_network.trainable = False
# embedding_network.summary()

In [67]:
inputx = tf.keras.Input(shape=IMG_SHAPE)
x = tf.keras.layers.BatchNormalization()(inputx)
x = tf.keras.layers.Conv2D(4, (5, 5), activation="tanh")(x)
x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(16, (5, 5), activation="tanh")(x)
x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(output_classes, activation="tanh")(x)
embedding_network = tf.keras.Model(inputx, x)

In [68]:
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

In [69]:
input_1 = tf.keras.layers.Input(IMG_SHAPE)
input_2 = tf.keras.layers.Input(IMG_SHAPE)

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = tf.keras.layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)

In [70]:
def loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

  Arguments:
      margin: Integer, defines the baseline for distance for which pairs
              should be classified as dissimilar. - (default is 1).

  Returns:
      'constrastive_loss' function with data ('margin') attached.
  """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.

      Arguments:
          y_true: List of labels, each label is of type float32.
          y_pred: List of predictions of same length as of y_true,
                  each label is of type float32.

      Returns:
          A tensor containing constrastive loss as floating point value.
      """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

In [71]:
siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
siamese.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
functional_5 (Functional)       (None, 197)          11968865    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           functional_5[0][0]    

In [72]:
history = siamese.fit(
    [x_train_1, x_train_2],
    labels_train,
    validation_data=([x_val_1, x_val_2], labels_val),
    batch_size=batch_size,
    epochs=epochs,
)

Epoch 1/10
247/247 [==============================] - 177s 718ms/step - loss: 0.0759 - accuracy: 0.9155 - val_loss: 0.0592 - val_accuracy: 0.9220
Epoch 2/10
247/247 [==============================] - 173s 702ms/step - loss: 0.0618 - accuracy: 0.9254 - val_loss: 0.0555 - val_accuracy: 0.9200
Epoch 3/10
247/247 [==============================] - 174s 703ms/step - loss: 0.0531 - accuracy: 0.9332 - val_loss: 0.0531 - val_accuracy: 0.9280
Epoch 4/10
247/247 [==============================] - 175s 707ms/step - loss: 0.0477 - accuracy: 0.9431 - val_loss: 0.0509 - val_accuracy: 0.9360
Epoch 5/10
247/247 [==============================] - 168s 679ms/step - loss: 0.0440 - accuracy: 0.9449 - val_loss: 0.0571 - val_accuracy: 0.9180
Epoch 6/10
247/247 [==============================] - 167s 676ms/step - loss: 0.0488 - accuracy: 0.9381 - val_loss: 0.0572 - val_accuracy: 0.9260
Epoch 7/10
247/247 [==============================] - 167s 677ms/step - loss: 0.0480 - accuracy: 0.9396 - val_loss: 0.0468 -

In [73]:
def plt_metric(history, metric, title, has_valid=True):
    """Plots the given 'metric' from 'history'.

    Arguments:
        history: history attribute of History object returned from Model.fit.
        metric: Metric to plot, a string value present as key in 'history'.
        title: A string to be used as title of plot.
        has_valid: Boolean, true if valid data was passed to Model.fit else false.

    Returns:
        None.
    """
    plt.plot(history[metric])
    if has_valid:
        plt.plot(history["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()


# Plot the accuracy
plt_metric(history=history.history, metric="accuracy", title="Model accuracy")

# Plot the constrastive loss
plt_metric(history=history.history, metric="loss", title="Constrastive Loss")

In [74]:
results = siamese.evaluate([x_val_1, x_val_2], labels_val) #suppose is x_test_1
print("test loss, test acc:", results)

16/16 [==============================] - 3s 185ms/step - loss: 0.0679 - accuracy: 0.9100
test loss, test acc: [0.0679105669260025, 0.9100000262260437]


In [75]:
def visualize(pairs, labels, to_show=6, num_col=3, predictions=None, test=False):
    """Creates a plot of pairs and labels, and prediction if it's test dataset.

    Arguments:
        pairs: Numpy Array, of pairs to visualize, having shape
               (Number of pairs, 2, 28, 28).
        to_show: Int, number of examples to visualize (default is 6)
                `to_show` must be an integral multiple of `num_col`.
                 Otherwise it will be trimmed if it is greater than num_col,
                 and incremented if if it is less then num_col.
        num_col: Int, number of images in one row - (default is 3)
                 For test and train respectively, it should not exceed 3 and 7.
        predictions: Numpy Array of predictions with shape (to_show, 1) -
                     (default is None)
                     Must be passed when test=True.
        test: Boolean telling whether the dataset being visualized is
              train dataset or test dataset - (default False).

    Returns:
        None.
    """

    num_row = to_show // num_col if to_show // num_col != 0 else 1

    # `to_show` must be an integral multiple of `num_col`
    #  we found num_row and we have num_col
    #  to increment or decrement to_show
    #  to make it integral multiple of `num_col`
    #  simply set it equal to num_row * num_col
    to_show = num_row * num_col
    # Plot the images
    fig, axes = plt.subplots(num_row, num_col, figsize=(5, 5))
    for i in range(to_show):

        # If the number of rows is 1, the axes array is one-dimensional
        if num_row == 1:
            ax = axes[i % num_col]
        else:
            ax = axes[i // num_col, i % num_col]
        ax.imshow(tf.concat([pairs[i][0].astype(np.uint8), pairs[i][1].astype(np.uint8)], axis=1))

        # ax.imshow(np.squeeze([pairs[i][0]], axis=0))

        ax.set_axis_off()
        if test:
            ax.set_title("True: {} | Pred: {:.5f}".format(labels[i], predictions[i][0]))
        else:
            ax.set_title("Label: {}".format(labels[i]))
    if test:
        plt.tight_layout(rect=(0, 0, 1.9, 1.9), w_pad=0.0)
    else:
        plt.tight_layout(rect=(0, 0, 1.5, 1.5))
    plt.show()

In [76]:
predictions = siamese.predict([x_val_1, x_val_2])

In [77]:
visualize(pairs_val, labels_val, to_show=9, predictions=predictions, test=True)